<a href="https://colab.research.google.com/github/rogerb831/Text2LIVE/blob/main/Text2LIVE_Toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Optional, mount Dropbox
!wget https://downloads.rclone.org/v1.53.2/rclone-v1.53.2-linux-amd64.deb
!apt install ./rclone-v1.53.2-linux-amd64.deb
!rclone config
!sudo mkdir /content/dropbox
!nohup rclone --vfs-cache-mode writes mount dropbox:/stable_diffusion /content/dropbox &

In [ ]:
#@title Optional, mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

# Set Variables & Install Dependencies

In [ ]:
import cv2
import os

#user inputs
toolkit_path="/content/AI_VFX_Toolkit/data" #@param {type:"string"}
input_video_path= "/content/gdrive/MyDrive/AI_VFX_Toolkit/il-short.mp4" #@param {type:"string"}


# Assume some paths to simplify user experience
frames_dir = toolkit_path +'/frames/'
flow_dir = toolkit_path +'/frames_flow/'
maskrcnn_dir = toolkit_path +'/frames_maskrcnn/'

#Make sure dirs exist
alldirs = [frames_dir, flow_dir, maskrcnn_dir]
for dir in alldirs:
  if not os.path.exists(dir):
   os.makedirs(dir, mode=0o777, exist_ok=False)


#Install deps
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

!pip install --upgrade --no-cache-dir gdown
!git clone https://github.com/rogerb831/layered-neural-atlases.git
!git clone https://github.com/rogerb831/Text2LIVE.git -b Colab-Enhancements
%cd /content/layered-neural-atlases
!pip install -r requirements.txt
!git submodule update --init
!cd thirdparty/RAFT/; sh ./download_models.sh
%cd /content/Text2LIVE
!sed -i '/torch/d' requirements.txt #Hack for colab compatibility
!pip install -r requirements.txt

In [ ]:
#@title Extract frames from video

video = cv2.VideoCapture(input_video_path)
success = True
count = 1
while success:
  success,frame = video.read()
  #
  name = frames_dir+str(count).zfill(5)+'.jpg'
  if success == True:
    cv2.imwrite(name,frame)
    print('Frame {} Extracted Successfully'.format(count))#, end='\r')
    count = count+1
  else:
    break



In [ ]:
#@title Create Masks

#@markdown You need to define a class which will be used to identify the foreground object from the background. [Click here](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/) for a full list of classes
class_name= "person" #@param {type:"string"}
%cd /content/layered-neural-atlases
print("frames_dir = " +frames_dir)
print("class_name = " +class_name)
!python preprocess_mask_rcnn.py --vid-path {frames_dir} --class_name {class_name}

In [ ]:
#@title Create Optical Flows 
#@markdown This step Requires around 40GB of GPU memeory (premium colab GPU). It creates an npy file for each frame transition, which are around 16Mb per frame. Be sure you have the available disk space and GPU memory or it will fail.
%cd /content/layered-neural-atlases
max_long_edge = round(video.get(cv2.CAP_PROP_FRAME_WIDTH))

#max_long_edge = 768 #@param {type:"string"}
!python preprocess_optical_flow.py --vid-path {frames_dir} --max_long_edge {max_long_edge}

# Train Custom Video
Training consists of two step.


1.   Create a pretained model using layered-neural-atlases
2.   Train the final output using the pretrained model and Test2LIVE



In [ ]:
#@title Pretrail Custom model
%cd /content/layered-neural-atlases
!python train.py config/config.json

In [ ]:
#@title Test2Live trainin
#@markdown Under development. If you want to test training, use the exmaple block below.


# Train example video

Run this block if you want to train using the demo pretrained model against the demo videos.

In [ ]:
#@title Download model & sample data
%cd /content/Text2LIVE
!gdown --fuzzy -O data.zip "https://drive.google.com/uc?id=1osN4PlPkY9uk6pFqJZo8lhJUjTIpa80J&export=download";
!unzip data.zip

In [ ]:
!python train_video.py --example_config car-turn_winter.yaml